## Ingest images

In [ ]:
from buck.analysis.basics import ingest_images

fpath = "..\\images\\squared\\*.png"
images,ages = ingest_images(fpath)

## Split datasets

In [ ]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

## Homogenize data across classes

In [ ]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 10
X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

## Build Conv. Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.optimizers import Adam

# Create a simpler model that's less likely to overfit
model = Sequential()

# First convolutional block - keep it simple
model.add(Conv2D(8, kernel_size=3, padding='same', activation='relu', input_shape=(288, 288, 1)))
model.add(MaxPooling2D(pool_size=2))

# Second convolutional block
model.add(Conv2D(16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

# Third convolutional block
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

# Global pooling to reduce parameters
model.add(GlobalAveragePooling2D())

# Classification head - minimal dense layers
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Moderate dropout
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# Use a much lower learning rate - crucial for small datasets
model.compile(
    loss='categorical_crossentropy', 
    optimizer=Adam(learning_rate=0.00005),  # Very low learning rate
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights properly based on the raw class distribution
# Extract the class labels from one-hot encoded y_train_orig
y_integers = np.argmax(y_train_flat, axis=1)

# Compute balanced class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_integers),
    y=y_integers
)

# Convert to dictionary format for Keras
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("Class weights:", class_weight_dict)

# Set up improved callbacks
checkpointer = ModelCheckpoint(
    filepath='model_improved.weights.best.hdf5.keras',
    verbose=1, 
    save_best_only=True,
    monitor='val_accuracy'  # Changed to monitor accuracy instead of loss
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,  # Give it more time to learn
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print("\nTraining with improved configuration:")
hist_improved = model.fit(
    X_train_pca, 
    y_train_flat,
    batch_size=8,  # Smaller batch size for better learning with small dataset
    epochs=100,
    validation_data=(Xval, yval),
    callbacks=[checkpointer, early_stopping, reduce_lr],
    verbose=1,
    shuffle=True,
    class_weight=class_weight_dict
)